In [1]:
import utils

In [2]:
LOWER = True
NORM_TENSE = True
REP_NUM = True
REP_YEAR = True
REP_SYM = False

In [3]:
dataset_folder = "data/EN/"
train_data = dataset_folder + "train"
lines = utils.read_file_to_lines(train_data)

In [4]:
sentences = []
pos_tags = []

while len(lines) > 1:
    sentence_break = lines.index("")
    sentence_xy = lines[:sentence_break]
    words = [utils.preprocess_text(token.split(" ")[0],
                                   lower=LOWER,
                                   norm_tense=NORM_TENSE,
                                   replace_number=REP_NUM,
                                   replace_year=REP_YEAR,
                                   replace_symbol=REP_SYM)
             for token in sentence_xy]
    pos = [token.split(" ")[1] for token in sentence_xy]
    sentences.append(words)
    pos_tags.append(pos)
    lines = lines[sentence_break+1:]

In [5]:
o_list = ["a", "the"]
np_list = []
vp_list = []

def get_entities(sentence, pos_tag):
    for i in range(len(sentence)):
        try:
            pos = pos_tag[i].split("-")[1]
            word = sentence[i]
            if word not in o_list and word[0].isalpha():
                if pos == "NP":
                    np_list.append(word)
                elif pos == "VP":
                    vp_list.append(word)
        except Exception as e:
            pass
        
for n in range(len(sentences)):
    sentence = sentences[n]
    pos_tag = pos_tags[n]
    get_entities(sentence, pos_tag)
    
def clean_list(input_list):
    input_list = list(set(input_list))
    input_list.sort()
    return input_list

np_list = clean_list(np_list)
len_np = len(np_list)

vp_list = clean_list(vp_list)
len_vp = len(vp_list)

In [6]:
import copy
from random import randint, choice

In [10]:
def rand_bool():
    return choice([True, False])

def augment_line(sentence, pos_tag):
    start = False
    for i, word in enumerate(sentence):
        try:
            b, pos = pos_tag[i].split("-")
            if pos == "NP" and sentence[i] not in o_list:
                if rand_bool():
                    r_i = randint(0, len_np)
                    sentence[i] = np_list[r_i]
            elif pos == "VP" and sentence[i] not in o_list:
                if rand_bool():
                    r_i = randint(0, len_vp)
                    sentence[i] = vp_list[r_i]
        except:
            pass
    return sentence

In [11]:
aug_lines = []

#for n in range(len(sentences)):
for n in range(20):
    sentence = sentences[n]
    sentence = copy.deepcopy(sentence)
    print("org:", " ".join(sentence))
    pos_tag = pos_tags[n]
    sentence = augment_line(sentence, pos_tag)
    for i, word in enumerate(sentence):
        pos = pos_tag[i]
        line = word + " " + pos
        aug_lines.append(line)
    print("aug:", " ".join(sentence))
    print("")

org: municipal bond are generally a bit safer than corporate bond in a recession , but not as safe as bond issued by the federal government .
aug: army bond rest generally a bit safer than corporate bond in a recession , but not as safe as previously issued by the federal government .

org: he added that the cost for stress-related compensation claim is about twice the average for all injury claim .
aug: he added that the cost for stress-related gorman circuit is about refuge the epicenter for all injury commerciale .

org: the incident occurred saturday night .
aug: the dealing toiling saturday night .

org: he said : `` we know of no technology that exist anywhere in the world that would allow u to '' reinforce the column .
aug: he said : `` we withstand of usa technology contradictory commit anywhere in the world that finishing expose mounting to '' reinforce the are .

org: walter h. monteith jr. , snet chairman and chief executive officer , said : `` innovative marketing of our pr

In [9]:
#aug_lines[:100]